In [1]:
import pandas as pd
import json
import re
import numpy as np

In [2]:
lake = pd.read_csv("Resources/lake_info/lake_information.csv", converters={"Lake ID": str})
length = pd.read_csv("Resources/fish_length_info/fish_length.csv", converters={"lake_ID": str, "survey_ID": str})
catch = pd.read_csv("Resources/fish_catch_info/fish_catch.csv", converters={"lake_ID": str, "survey_ID": str})
was = pd.read_csv("Resources/was_info/water_access_information.csv")

In [3]:
#add methods
#sort and format year survey data then convert to datetime
def survey_date_cleaner(dfseries):
    dfseries = dfseries.copy()
    for _ in range(len(dfseries)):
        dfseries[_] = dfseries[_].replace("/","-")
        if matches := re.search(r"^(\d?\d{1})-(\d?\d{1})-(\d{4})$", dfseries[_], re.IGNORECASE):
            dfseries[_] = f"{matches.group(3)}-{matches.group(2)}-{matches.group(1)}"
        try:
            dfseries[_] = pd.to_datetime(dfseries[_],format="%Y-%m-%d")
        except ValueError:
            if matches := re.search(r"^(\d{4})-(\d?\d{1})-(\d?\d{1})$", dfseries[_], re.IGNORECASE):
                dfseries[_] = f"{matches.group(1)}-{matches.group(3)}-{matches.group(2)}"
            dfseries[_] = pd.to_datetime(dfseries[_],format="%Y-%m-%d")
    dfseries = pd.to_datetime(dfseries,format="%Y-%m-%d")
    return dfseries

#input CPUE column data series
#drop strings from cpue data series and convert to int(found some "? marks")
def cpue_cleaner(dfseries):
    i=0
    for _ in dfseries:
        try:
            np.float64(_)
            i = i + 1
        except ValueError:
            print(i, _,"valueerror")
            dfseries = dfseries.drop([i])
            i = i + 1
    dfseries = pd.to_numeric(dfseries)
    return dfseries

#input fish_count column data series
#returns fish_count column in dataframe as a list of individual integer measurements e.g. [0,0,1,1,1,2,3,3,4]
def fish_length_cleaner(dfseries):
    fish_lengths = []
    #scrub each row string and convert to list of lengths sampled
    for row in dfseries:
        #clean up string of list syntax
        sample = row.replace("]","").replace("[","").replace(" ","").split(",")
        #breakdown string into list of lengths sampled for row
        sample_lengths = []
        for n in range(len(sample)):
            if n % 2 == 1:
                fishlength = int(sample[n-1])
                fishcount = int(sample[n])
                for c in range(fishcount):
                    sample_lengths.append(fishlength)
        fish_lengths.append(sample_lengths)
    return fish_lengths

#input string data series with rows of "[1,1,2,2,2,3,4,7,7,7]"
#returns fish_count series column as a list of float averages e.g. [5.1, 4.6, 8.8]
def fish_length_averager(dfseries):
    fish_length_averages = []
    for row in dfseries:
        sample = [int(x) for x in row if x != 0]
        if bool(sample) == True:
            fish_length_averages.append(np.mean(sample))
        else:
            fish_length_averages.append(np.nan)
    return fish_length_averages

#input quartile data series
#split lower quartile and convert to float
def lower_quartile_cleaner(dseries):
    quartile_list = []
    for quartile in dseries:
        try:
            if len(quartile.split("-")) == 2:
                try:
                    quartile_list.append(float(quartile.split("-")[0]))
                except ValueError:
                    quartile_list.append(float("nan"))
                    pass
            else:
                quartile_list.append(float("nan"))
        except AttributeError:
            quartile_list.append(quartile)
            pass
    return quartile_list

#input quartile data series
#split upper quartile and convert to float
def upper_quartile_cleaner(dseries):
    quartile_list = []
    for quartile in dseries:
        try:
            if len(quartile.split("-")) == 2:
                try:
                    quartile_list.append(float(quartile.split("-")[1]))
                except ValueError:
                    quartile_list.append(float("nan"))
                    pass
            else:
                quartile_list.append(float("nan"))
        except AttributeError:
            quartile_list.append(quartile)
            pass
    return quartile_list



In [4]:
lake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6011 entries, 0 to 6010
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Lake ID             6011 non-null   object 
 1   Lake Name           6011 non-null   object 
 2   Lake County         6011 non-null   object 
 3   Nearest Town        5377 non-null   object 
 4   Lake Coordinates    6011 non-null   object 
 5   Lake Area           6011 non-null   float64
 6   Littoral Area       6011 non-null   float64
 7   Lake Depth          6011 non-null   float64
 8   Mean Depth          6011 non-null   float64
 9   Water Access Sites  6011 non-null   object 
dtypes: float64(4), object(6)
memory usage: 469.7+ KB


In [5]:
lake = lake.drop_duplicates()
lake_coordinates = [json.loads(x) for x in lake["Lake Coordinates"]]
longitude = [x[0] for x in lake_coordinates]
latitude = [x[1] for x in lake_coordinates]
lake["Longitude"] = longitude
lake["Latitude"] = latitude
lake = lake.drop(columns=["Lake Coordinates"])
lake.head()

,Lake ID,Lake Name,Lake County,Nearest Town,Lake Area,Littoral Area,Lake Depth,Mean Depth,Water Access Sites,Longitude,Latitude
0,01004000,Aitkin,Aitkin,Libby,658.82,453.0,35.0,0.0,[],-93.289871,46.804042
1,01003100,Anderson,Aitkin,Sheshebee,91.34,83.0,16.0,3.0,[],-93.232645,46.737721
2,01016900,Bachelor,Aitkin,Aitkin,53.56,30.0,47.0,12.0,[],-93.726840,46.473611
3,01004600,Ball Bluff,Aitkin,Jacobson,168.64,32.0,78.0,25.0,['was01599'],-93.280365,46.932611
4,01006300,Bass,Aitkin,McGregor,89.13,55.0,42.0,0.0,[],-93.310700,46.808330


In [6]:
lake.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5422 entries, 0 to 6009
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Lake ID             5422 non-null   object 
 1   Lake Name           5422 non-null   object 
 2   Lake County         5422 non-null   object 
 3   Nearest Town        5008 non-null   object 
 4   Lake Area           5422 non-null   float64
 5   Littoral Area       5422 non-null   float64
 6   Lake Depth          5422 non-null   float64
 7   Mean Depth          5422 non-null   float64
 8   Water Access Sites  5422 non-null   object 
 9   Longitude           5422 non-null   float64
 10  Latitude            5422 non-null   float64
dtypes: float64(6), object(5)
memory usage: 508.3+ KB


In [7]:
was.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Site ID                      3002 non-null   object
 1   Site Name                    3002 non-null   object
 2   Directions to Site           3002 non-null   object
 3   Site Coordinates             3002 non-null   object
 4   Administrator of Site        2999 non-null   object
 5   Number of Docks              3002 non-null   int64 
 6   Number of Restrooms          3002 non-null   int64 
 7   Number of Parking Lots       3002 non-null   int64 
 8   Number of Accessible Spaces  3002 non-null   int64 
 9   Number of Trailer Spaces     3002 non-null   int64 
 10  Number of Vehicle Spaces     3002 non-null   int64 
 11  Parking Lot Surface          2706 non-null   object
 12  Number of Ramps              3002 non-null   int64 
 13  Ramp Surface                 2918

In [8]:
was = was.drop_duplicates()
site_coordinates = [json.loads(x) for x in was["Site Coordinates"]]
longitude = [x[0] for x in site_coordinates]
latitude = [x[1] for x in site_coordinates]
was["Longitude"] = longitude
was["Latitude"] = latitude
was = was.drop(columns=["Site Coordinates"])
was.head()

,Site ID,Site Name,Directions to Site,Administrator of Site,Number of Docks,Number of Restrooms,Number of Parking Lots,Number of Accessible Spaces,Number of Trailer Spaces,Number of Vehicle Spaces,Parking Lot Surface,Number of Ramps,Ramp Surface,Longitude,Latitude
0,WAS00001,South Fork Kawishiwi River (S) Public Water Ac...,"From Ely drive 10.4 miles south on Hwy 1, turn...",U.S. Forest Service,1,1,1,0,7,7,gravel,1,concrete slab,-91.733457,47.816558
1,WAS00002,"Rainy Lake, Ranier Public Water Access Site",In the City of Ranier; launch ramp is Spruce S...,City of Ranier,1,0,1,0,8,0,NaN,1,concrete slab,-93.348122,48.615995
2,WAS00003,Severance Lake Public Water Access Site,Head east out of New Auburn for 7 miles to the...,DNR Division of Fish and Wildlife,0,0,0,0,0,0,NaN,0,NaN,-94.132386,44.675798
3,WAS00004,"Redwood River, Wabasso Public Water Access Site",On Hwy 6 between 325th St and 310th St,DNR Division of Parks and Trails,0,0,1,0,10,0,natural,1,concrete slab,-95.229799,44.507897
4,WAS00005,Shetek Lake Marsh's Landing Public Water Acces...,From US-59 follow Valhalla Rd east for 1.5 mil...,Murray County,2,1,1,0,15,5,gravel,2,concrete slab,-95.713395,44.108909


In [9]:
was.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3002 entries, 0 to 3001
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Site ID                      3002 non-null   object 
 1   Site Name                    3002 non-null   object 
 2   Directions to Site           3002 non-null   object 
 3   Administrator of Site        2999 non-null   object 
 4   Number of Docks              3002 non-null   int64  
 5   Number of Restrooms          3002 non-null   int64  
 6   Number of Parking Lots       3002 non-null   int64  
 7   Number of Accessible Spaces  3002 non-null   int64  
 8   Number of Trailer Spaces     3002 non-null   int64  
 9   Number of Vehicle Spaces     3002 non-null   int64  
 10  Parking Lot Surface          2706 non-null   object 
 11  Number of Ramps              3002 non-null   int64  
 12  Ramp Surface                 2918 non-null   object 
 13  Longitude         

In [10]:
length.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100688 entries, 0 to 100687
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   lake_ID         100688 non-null  object
 1   species         100688 non-null  object
 2   fish_count      100688 non-null  object
 3   maximum_length  100688 non-null  int64 
 4   minimum_length  100688 non-null  int64 
 5   survey_ID       100688 non-null  object
 6   survey_date     100688 non-null  object
dtypes: int64(2), object(5)
memory usage: 5.4+ MB


In [11]:
length = length.drop_duplicates()
length["survey_date"] = survey_date_cleaner(length["survey_date"])
length["fish_count"] = fish_length_cleaner(length["fish_count"])
length["averageLength"] = fish_length_averager(length["fish_count"])
length = length.sort_values("survey_date")
length.head()

,lake_ID,species,fish_count,maximum_length,minimum_length,survey_ID,survey_date,averageLength
61824,62006700,BIB,[0],0,0,120236.0,1953-09-01,NaN
61823,62006700,GSF,[0],0,0,120236.0,1953-09-01,NaN
61825,62006700,WTS,[0],0,0,120236.0,1953-09-01,NaN
61822,62006700,BLG,[0],0,0,120236.0,1953-09-01,NaN
61820,62006700,CAP,[0],0,0,120236.0,1953-09-01,NaN


In [12]:
length.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100688 entries, 61824 to 36869
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   lake_ID         100688 non-null  object        
 1   species         100688 non-null  object        
 2   fish_count      100688 non-null  object        
 3   maximum_length  100688 non-null  int64         
 4   minimum_length  100688 non-null  int64         
 5   survey_ID       100688 non-null  object        
 6   survey_date     100688 non-null  datetime64[ns]
 7   averageLength   100500 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 6.9+ MB


In [13]:
catch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311211 entries, 0 to 311210
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   lake_ID         311211 non-null  object 
 1   CPUE            311211 non-null  object 
 2   averageWeight   311195 non-null  float64
 3   gear            311211 non-null  object 
 4   gearCount       311211 non-null  float64
 5   quartileCount   202694 non-null  object 
 6   quartileWeight  200945 non-null  object 
 7   species         311211 non-null  object 
 8   survey_ID       311211 non-null  object 
 9   survey_date     311211 non-null  object 
 10  totalCatch      311211 non-null  int64  
 11  totalWeight     311211 non-null  float64
dtypes: float64(3), int64(1), object(8)
memory usage: 28.5+ MB


In [14]:
catch = catch.drop_duplicates()
catch["CPUE"] = cpue_cleaner(catch["CPUE"])
catch["survey_date"] = survey_date_cleaner(catch["survey_date"])
catch["count_lower_quartile"] = lower_quartile_cleaner(catch["quartileCount"])
catch["count_upper_quartile"] = upper_quartile_cleaner(catch["quartileCount"])
catch["weight_lower_quartile"] = lower_quartile_cleaner(catch["quartileWeight"])
catch["weight_upper_quartile"] = upper_quartile_cleaner(catch["quartileWeight"])
catch["gearCount"] = catch["gearCount"].astype("int")
catch = catch.drop(columns=["quartileCount", "quartileWeight"])
catch = catch.sort_values("survey_date")

2298 ? valueerror
2311 ? valueerror
2321 ? valueerror
2328 ? valueerror
2348 ? valueerror
2701 ? valueerror
4772 ? valueerror
5387 ? valueerror
9361 ? valueerror
10210 ? valueerror
11307 ? valueerror
14677 ? valueerror
14678 ? valueerror
14831 ? valueerror
14833 ? valueerror
14841 ? valueerror
14880 ? valueerror
14894 ? valueerror
14899 ? valueerror
14902 ? valueerror
14905 ? valueerror
14907 ? valueerror
15018 ? valueerror
15469 ? valueerror
15484 ? valueerror
33692 ? valueerror
33695 ? valueerror
33705 ? valueerror
33707 ? valueerror
46874 ? valueerror
48269 ? valueerror
49482 ? valueerror
49632 ? valueerror
50753 ? valueerror
60069 ? valueerror
64867 ? valueerror
64869 ? valueerror
64873 ? valueerror
64875 ? valueerror
64876 ? valueerror
64878 ? valueerror
64879 ? valueerror
64882 ? valueerror
64883 ? valueerror
72410 ? valueerror
82630 ? valueerror
82756 ? valueerror
82788 ? valueerror
94545 ? valueerror
94547 ? valueerror
94556 ? valueerror
94560 ? valueerror
94566 ? valueerror
94

In [15]:
catch.head()

,lake_ID,CPUE,averageWeight,gear,gearCount,species,survey_ID,survey_date,totalCatch,totalWeight,count_lower_quartile,count_upper_quartile,weight_lower_quartile,weight_upper_quartile
53927,10004200,1.75,3.64,Standard trap nets,4,NOP,121577.0,1940-06-24,7,25.50,NaN,NaN,NaN,NaN
53934,10004200,16.50,0.15,Standard gill nets,2,YEP,121577.0,1940-06-24,33,4.80,2.0,22.3,0.1,0.2
53935,10004200,0.25,1.25,Standard trap nets,4,LMB,121577.0,1940-06-24,1,1.25,0.2,0.8,0.3,1.5
53936,10004200,1.00,0.11,Standard trap nets,4,PMK,121577.0,1940-06-24,4,0.45,0.7,6.5,0.1,0.2
53937,10004200,21.00,0.20,Standard trap nets,4,BLG,121577.0,1940-06-24,84,16.50,3.5,57.1,0.1,0.3


In [16]:
catch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311211 entries, 53927 to 195323
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   lake_ID                311211 non-null  object        
 1   CPUE                   311012 non-null  float64       
 2   averageWeight          311195 non-null  float64       
 3   gear                   311211 non-null  object        
 4   gearCount              311211 non-null  int32         
 5   species                311211 non-null  object        
 6   survey_ID              311211 non-null  object        
 7   survey_date            311211 non-null  datetime64[ns]
 8   totalCatch             311211 non-null  int64         
 9   totalWeight            311211 non-null  float64       
 10  count_lower_quartile   202050 non-null  float64       
 11  count_upper_quartile   202050 non-null  float64       
 12  weight_lower_quartile  200365 non-null  

In [19]:
cpue_median = catch.groupby(["lake_ID", "species"]).median() ["CPUE"]
cpue_median

lake_ID   species
01000100  BCS         0.330
          BKF         1.670
          BLB         0.855
          BLC         1.555
          BLG         7.280
                      ...  
87003000  OSS        15.055
          TME         0.360
          WAE         9.500
          WTS         0.110
          YEP         2.500
Name: CPUE, Length: 41968, dtype: float64

In [22]:
lake.to_csv("Resources/lake_info/lake_information_clean.csv", index=False)
length.to_csv("Resources/fish_length_info/fish_length_clean.csv", index=False)
catch.to_csv("Resources/fish_catch_info/fish_catch_clean.csv", index=False)
was.to_csv("Resources/was_info/water_access_information_clean.csv", index=False)
cpue_median.to_csv("Resources/fish_catch_info/cpue_median_clean.csv", index=True)